In [1]:
import time
import matplotlib.pyplot as plt
import random
import numpy as np
import pickle

import os
from pathlib import Path

In [2]:
current_file_path = Path(get_ipython().run_line_magic('pwd', ''))
root_path = current_file_path.parent
while not any(file.suffix == ".ipynb" for file in root_path.glob("*")):
    root_path = root_path.parent

root_path = str(root_path)
print(root_path)

from keras.models import load_model
model_dir='06_04_230702'#xx_xx_xxxxxx
model=load_model(os.path.join(root_path,'model_training','model_record',model_dir,'model.h5'))



c:\Users\88692\Desktop\model_pre_train


In [3]:
import sys
import random
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QWidget, QGridLayout, QMessageBox
from PyQt5.QtGui import QPainter, QPen, QColor, QFont, QPainterPath
from PyQt5.QtCore import QTimer, Qt, QPoint
import numpy as np
import pickle
from pathlib import Path
import os
import math
# probs=np.full((2, 225),0)
probs = [0 for _ in range(225)]
a=[False]

class DrawingWidget(QWidget):
    
    def __init__(self, board_size):
        super().__init__()  
        self.board_size = board_size
        self.show_AI_ana=True
    def paintEvent(self, event):
        super().paintEvent(event) 
        painter = QPainter(self)
        pen = QPen(Qt.black, 1) 
        for col in range(15):
            painter.setPen(pen)
            painter.drawLine(29+col*40, 24, 29+col*40, 584)

        for row in range(15):
            painter.setPen(pen)
            painter.drawLine(29, 24+row*40, 589, 24+row*40)

        # 繪制O色圆形
        pen = QPen(Qt.black)
        painter.setPen(pen)
        painter.setBrush(Qt.black)
        radius = 5
        
        center1 = QPoint(149, 145)
        center2 = QPoint(469, 145)
        center3 = QPoint(309, 304)
        center4 = QPoint(149, 465)
        center5 = QPoint(469, 465)
        for center in [center1,center2,center3,center4,center5]:
            painter.drawEllipse(center, radius, radius)

        painter2=QPainter(self)
        painter2.setRenderHint(QPainter.Antialiasing) 
        painter2.setRenderHint(QPainter.Antialiasing, True)
        painter2.setPen(QPen(Qt.NoPen))
        for row in range(self.board_size):
            for col in range(self.board_size):
                opa=probs[row*15+col]
                
                if(a[0]):
                    painter2.setOpacity(math.pow(opa,0.3))
                    painter2.setBrush(QColor(20, 100, 255))
                    rect=painter2.drawRect(row*40+15, col*40+9, 30, 30)
class MainWindow(QMainWindow):
    def get_board(self):
        return self.board
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Gomoku')
        self.board_size = 15
        self.ai_player = 'O'
        self.board = [[-1] * self.board_size for _ in range(self.board_size)]
        self.game_over = False
        self.player_can_play = True 
        self.counter=0
        self.step_cnt=0
        self.state_record=np.full((15,15),-1)
        self.init_ui(startup=True)
        width = 670
        height = 700
        self.resize(width, height)
    def init_ui(self,startup):
        central_widget = QWidget()
        layout = QGridLayout()
        central_widget.setLayout(layout)

        self.status_label = QLabel()
        layout.addWidget(self.status_label, 0, 0)

        buttonM = QPushButton("AI analysis")
        buttonM.clicked.connect(self.ai_show_clicked)
        buttonM.setFixedSize(80, 30)
        layout.addWidget(buttonM, 0, 1)
        button_reset = QPushButton("reset")
        button_reset.clicked.connect(self.reset)
        button_reset.setFixedSize(50, 30)
        layout.addWidget(button_reset, 0, 2)
        self.drawing_widget = DrawingWidget(self.board_size)
        layout.addWidget(self.drawing_widget, 1, 0, 1, 2)
        
        self.buttons = []
        button_size = 40
        for row in range(self.board_size):
            row_buttons = []
            for col in range(self.board_size):
                button = QPushButton(self.drawing_widget)
                button.setFixedSize(button_size, button_size)
                button.setStyleSheet("background-color: transparent; border: none;")
                button.move((row * button_size)+10, (col * button_size)+4 )
                font = QFont()
                font.setPointSize(20)
                button.setFont(font)
                if(self.board[row][col]=='O'):
                    button.setText("⚫")
                elif(self.board[row][col]=='X'):
                    button.setText("⚪")
                row_buttons.append(button)
            
            self.buttons.append(row_buttons)
            
        self.setCentralWidget(central_widget)
        self.setStyleSheet("background-color: #D2B48C;")
        if(startup):
            self.ai_play()
    def reset(self):
        self.board = [[-1] * self.board_size for _ in range(self.board_size)]
        
        self.game_over = False
        self.counter=0
        self.step_cnt=0
        self.state_record=np.full((15,15),-1)
        self.ai_player='O'
        self.update_prob()
        self.init_ui(startup=True)
    def ai_show_clicked(self):
        if(a[0]==False):
            a[0]=True
            self.update_prob()
        else:
            a[0]=False
            self.init_ui(startup=False)
    def update_prob(self):
        if(a[0]):
            available_positions = []
            for row in range(self.board_size):
                for col in range(self.board_size):
                    if self.board[row][col] != -1:
                        available_positions.append(row*15+col)
            # print("get_prob")
            _,_,p,_=self.get_ai_play(available_positions)
            for i in range(225):
                probs[i]=p[i]
            self.init_ui(startup=False)
    def get_ai_play(self,available_positions):
        state=self.state_record
        x=np.full((5,15,15),0)
        p1=(state+1)%2
        all_move=np.where(state<0,0,1)
        p2=all_move-p1
        if(int(np.max(state))%2==0):
            x[2]=p2
            x[3]=p1
        else:
            x[3]=p2
            x[2]=p1
            x[0]+=1
        lastmove = np.where(state==self.step_cnt-1)
        if(self.step_cnt!=0):
            x[4][lastmove]=1
        x[1]=1-x[3]-x[2]
        x= np.expand_dims(x, axis=0)
        
        prediction=model.predict_on_batch(x)
        prob=prediction[0].flatten()
        value=prediction[1]
        # print(x)
        #add noise
        noise=np.random.uniform(low=0.98,high=1.02,size=(225))
        prob[available_positions]=0
        prob=prob*noise
        move=np.argmax(prob)
        
        print(move)
        return int(move/15),move%15,prob,value
    def ai_play(self):
        available_positions = []
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.board[row][col] != -1:
                    available_positions.append(row*15+col)
        
        if len(available_positions)<225 and not self.game_over:
            print("get AI move")
            row, col ,_,value= self.get_ai_play(available_positions)
            self.status_label.setText(f'WR:{value}')
            # get AI move
            self.board[row][col] = self.ai_player
            self.state_record[row][col]=self.step_cnt
            self.step_cnt=self.step_cnt+1
            
            if self.ai_player == 'O':
                self.buttons[row][col].setText("⚫")     
            else:
                self.buttons[row][col].setText("⚪")
            self.player=self.ai_player
            if self.check_win(row, col):
                self.game_over = True
                print(f"{self.ai_player} won")
                self.status_label.setText(f'{self.ai_player} won！')
            else:
                self.update_prob()
                QTimer.singleShot(1000,self.ai_play)
                
                if(self.ai_player=='X'):
                    self.ai_player='O'
                else:
                    self.ai_player='X'
                
                
    def check_win(self, row, col):
        for c in range(self.board_size - 5):
            if self.board[row][c] == self.board[row][c+1] == self.board[row][c+2] == self.board[row][c+3] ==  self.board[row][c+4] == self.board[row][c+5] == self.player:
                return False
        for r in range(self.board_size - 5):
            if self.board[r][col] == self.board[r+1][col] == self.board[r+2][col] == self.board[r+3][col] == self.board[r+4][col] ==  self.board[r+5][col] == self.player:
                return False
        for i in range(self.board_size - 5):
            if row - i >= 0 and col - i >= 0 and row - i + 5 < self.board_size and col - i + 5 < self.board_size:
                if self.board[row-i][col-i] == self.board[row-i+1][col-i+1] == self.board[row-i+2][col-i+2] == self.board[row-i+3][col-i+3] == self.board[row-i+4][col-i+4] == self.board[row-i+5][col-i+5] == self.player:
                    return False
        for i in range(self.board_size - 5):
            if row + i < self.board_size and col - i >= 0 and row + i - 5 >= 0 and col - i + 5 < self.board_size:
                if self.board[row+i][col-i] == self.board[row+i-1][col-i+1] == self.board[row+i-2][col-i+2] == self.board[row+i-3][col-i+3] == self.board[row+i-4][col-i+4] == self.board[row+i-5][col-i+5] == self.player:
                    return False      
        for c in range(self.board_size - 4):
            if self.board[row][c] == self.board[row][c+1] == self.board[row][c+2] == self.board[row][c+3] == self.board[row][c+4] == self.player:
                return True
        for r in range(self.board_size - 4):
            if self.board[r][col] == self.board[r+1][col] == self.board[r+2][col] == self.board[r+3][col] == self.board[r+4][col] == self.player:
                return True
        for i in range(self.board_size - 4):
            if row - i >= 0 and col - i >= 0 and row - i + 4 < self.board_size and col - i + 4 < self.board_size:
                if self.board[row-i][col-i] == self.board[row-i+1][col-i+1] == self.board[row-i+2][col-i+2] == self.board[row-i+3][col-i+3] == self.board[row-i+4][col-i+4] == self.player:
                    return True
        for i in range(self.board_size - 4):
            if row + i < self.board_size and col - i >= 0 and row + i - 4 >= 0 and col - i + 4 < self.board_size:
                if self.board[row+i][col-i] == self.board[row+i-1][col-i+1] == self.board[row+i-2][col-i+2] == self.board[row+i-3][col-i+3] == self.board[row+i-4][col-i+4] == self.player:
                    return True
        return False


app = QApplication(sys.argv)
window = MainWindow()
window.show()
sys.exit(app.exec_())

get AI move
157
get AI move
171
get AI move
141
get AI move
125
get AI move
173
get AI move
189
get AI move
158
get AI move
186
get AI move
156
get AI move
155
get AI move
172
get AI move
187
get AI move
188


SystemExit: 0

c:\Users\88692\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
